In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os
import numpy as np

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '../testingdir/parameters.xml'

cfg = Parameters(config_file)
clim_estimation = cfg.get_clim_estimation()

radius=0.01
time_range=[cfg.start_time, cfg.end_time]
use_grid=True
cmap='terrain'
clim = [-1e3, 1e3]
use_cache = False
verbose = True
autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{t (ns)}$'
cbar_axis_label = r'$\frac{A}{m}$'
plot_name = r'$\mathbf{Magneitc\enspace Field\enspace Rotational\enspace Component}\enspace(H_{\phi})$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid, cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=verbose)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

In [ ]:
# get data
start_frame = end_frame = None
row_number = cfg.get_row_by_radius(radius)

data = []

# data = reader.get_frame_range_row('H_phi', row_number, start_frame, end_frame - 1)

for probe in cfg.probes:
    if (probe.type == 'row') and (probe.r_start == row_number):
        start_frame = start_frame or cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = end_frame or cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
        if probe.component == 'H_phi': data = reader.get_frame_range_row('H_phi', row_number, start_frame, end_frame)
        
if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        if (probe.type == 'frame') and (probe.r_start <= row_number) and (probe.z_start == 0) and (probe.r_end >= row_number) and (probe.z_end == cfg.number_z_grid):
            start_frame = start_frame or cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = end_frame or cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
            if probe.component == 'H_phi': 
                for piece in range(start_frame, end_frame, cfg.frames_per_file):
                    data[piece - start_frame:piece - start_frame + cfg.frames_per_file - 1] = reader.get_frame_range('H_phi', shape, piece, piece + cfg.frames_per_file)[:, row_number]

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
the_plot = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

In [ ]:
# add images
plot.add_image(plot_name, data, cmap=cmap, clim=clim)
plot.add_colorbar(plot_name, ticks=clim, title=cbar_axis_label)

plot.show()